In [1]:
import os
import torch
import timm
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from typing import Optional, Callable

from torch import nn, Tensor, optim
from torch.utils.data import Dataset, DataLoader
from torch.backends import cudnn
from torch import GradScaler
import torch.nn.functional as F

from torchvision.datasets import CIFAR100
from torchvision.transforms import v2
from sklearn.neighbors import NearestNeighbors

# ============ Configuration ============
config = {
    "dataset": "cifar100_noisy",
    "model": "resnet18",
    "pretrained": "imagenet",
    "epochs": 100,
    "batch_size": 128,
    "lr": 0.0005,
    "momentum": 0.9,
    "weight_decay": 0.05,
    "nesterov": True,
    "label_smoothing": 0.1,
    "optimizer": "adamw",
    "scheduler": "warm_restarts",     
    "warm_restarts_T0": 30,              
    "warm_restarts_mult": 1,             
    "cosine_eta_min": 1e-6,
    "scheduler_step_per_batch": True,
    "early_stop_patience": 35,
    "early_stop_mode": "max",
    "early_stop_min_delta": 0.05,
    "device": "cuda",
    "mixed_precision": True,
    "wandb_project": "cifar100-noisy-competition",
    "upscale_size": 224,  
    "aug_alpha": 0.5,          
    "cutmix_prob": 1.0,        
    "switch_epoch": 35,        
    "warmup_epochs": 10,
    "loss_threshold": 3.0,
    "dynamic_threshold_decay": 0.995,
    "soft_alpha": 0.4
}

device = torch.device(config["device"])
print(f"Using device: {device}")
cudnn.benchmark = True
pin_memory = True
enable_half = config["mixed_precision"]  # Disable for CPU, it is slower!
scaler = GradScaler(device, enabled=enable_half)

def get_refinement_metadata(dataset, device):
    print("\nExtracting Features for Analysis...")
    embed_model = timm.create_model('resnet18', pretrained=True, num_classes=0).to(device)
    embed_model.eval()
    
    analysis_transform = v2.Compose([
        v2.ToImage(), v2.Resize(224), v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    features, labels = [], np.array(dataset.targets)
    with torch.no_grad():
        for i in tqdm(range(len(dataset)), desc="Analyzing Data"):
            img, _ = dataset[i]
            img_t = analysis_transform(img).unsqueeze(0).to(device)
            features.append(embed_model(img_t).cpu().numpy())
    
    features = np.concatenate(features, axis=0)
    
    print("Running k-NN Agreement & Class Balancing...")
    K = 10
    knn = NearestNeighbors(n_neighbors=K+1, metric='cosine').fit(features)
    _, indices = knn.kneighbors(features)
    
    refurbished_labels = []
    agreement_scores = []
    for i in range(len(features)):
        neighbor_labels = labels[indices[i, 1:]]
        counts = np.bincount(neighbor_labels, minlength=100)
        refurbished_labels.append(np.argmax(counts))
        agreement_scores.append(counts[labels[i]] / K)
    
    agreement_scores = np.array(agreement_scores)
    refurbished_labels = np.array(refurbished_labels)

    stable_mask = agreement_scores >= 0.9
    counts_per_class = np.bincount(labels[stable_mask], minlength=100)
    rare_classes = np.where(counts_per_class < 20)[0] # Target classes below 20 samples
    
    for c in rare_classes:
        # Bolster rare classes using medium agreement samples (above 75%)
        potential_candidates = np.where((agreement_scores >= 0.75) & (refurbished_labels == c))[0]
        agreement_scores[potential_candidates] = 0.95 # Promote to "Stable" status
            
    del embed_model
    torch.cuda.empty_cache()
    
    return {"agreement": agreement_scores, "refurbished": refurbished_labels}

class SimpleCachedDataset(Dataset):
    def __init__(self, dataset):
        # Runtime transforms are not implemented in this simple cached dataset.
        self.data = tuple([x for x in dataset])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]

class PreprocessedDataset(Dataset):
    """
    Cache preprocessed tensors - apply transforms once and store results.
    
    PERFORMANCE OPTIMIZATION:
    - Applies deterministic transforms (ToImage, Resize) once at startup
    - Stores uint8 tensors (4x less memory than float32)
    - Random augmentations applied at runtime each epoch
    - Test set only needs normalization at runtime (huge speedup!)
    """
    def __init__(self, dataset, transform):
        print(f"Preprocessing {len(dataset)} images (this happens once)...")
        self.data = []
        self.targets = []
        
        for img, target in tqdm(dataset, desc="Caching", leave=False):
            transformed = transform(img)
            self.data.append(transformed)
            self.targets.append(target)
        
        print(f"Cached {len(self.data)} preprocessed images")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        return self.data[i], self.targets[i]

class RefinedAugmentationWrapper(Dataset):
    def __init__(self, preprocessed_dataset, runtime_transforms, refinement_map=None, alpha=0.4):
        self.dataset = preprocessed_dataset
        self.runtime_transforms = runtime_transforms
        self.refinement_map = refinement_map
        self.alpha = alpha
        
    def __len__(self):
        return len(self.dataset)
        
    def __getitem__(self, i):
        img_tensor, original_target = self.dataset[i]
        if self.runtime_transforms is not None:
            img_tensor = self.runtime_transforms(img_tensor)
        
        # Default: Hard label, full weight
        target_a = original_target
        target_b = original_target
        lam_target = 1.0
        weight = 1.0 

        if self.refinement_map is not None:
            score = self.refinement_map["agreement"][i]
            refurbished = int(self.refinement_map["refurbished"][i])
            
            # SOFT LABELING LOGIC
            # If k-NN and Original disagree (Noise/Overlap), we blend them.
            if score < 0.7:
                target_a = original_target
                target_b = refurbished
                lam_target = 1.0 - self.alpha  # e.g., 0.6 original, 0.4 refurbished
                weight = max(score, 0.4)       # Loss is scaled by k-NN confidence
            else:
                # Highly stable samples
                target_a = original_target
                target_b = original_target
                lam_target = 1.0
                weight = 1.0
                
        # We return two targets and a lambda to handle the "Soft Label" in the loss function
        return img_tensor, target_a, target_b, lam_target, weight

class CIFAR100_noisy_fine(Dataset):
    """
    See https://github.com/UCSC-REAL/cifar-10-100n, https://www.noisylabels.com/ and `Learning with Noisy Labels
    Revisited: A Study Using Real-World Human Annotations`.
    """

    def __init__(
        self, root: str, train: bool, transform: Optional[Callable], download: bool
    ):
        cifar100 = CIFAR100(
            root=root, train=train, transform=None, download=download
        )
        data, targets = tuple(zip(*cifar100))

        if train:
            noisy_label_file = os.path.join(root, "CIFAR-100-noisy.npz")
            if not os.path.isfile(noisy_label_file):
                raise FileNotFoundError(
                    f"{type(self).__name__} need {noisy_label_file} to be used!"
                )

            noise_file = np.load(noisy_label_file)
            if not np.array_equal(noise_file["clean_label"], targets):
                raise RuntimeError("Clean labels do not match!")
            targets = noise_file["noisy_label"]

        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, i: int):
        return self.data[i], self.targets[i]


class EarlyStopping:
    """Early stopping to stop training when validation metric doesn't improve."""
    def __init__(self, patience=10, min_delta=0.0, mode='max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_epoch = 0
        
    def __call__(self, score, epoch):
        if self.best_score is None:
            self.best_score = score
            self.best_epoch = epoch
            return False
        
        if self.mode == 'max':
            # For accuracy (higher is better)
            if score > self.best_score + self.min_delta:
                self.best_score = score
                self.best_epoch = epoch
                self.counter = 0
            else:
                self.counter += 1
        else:
            # For loss (lower is better)
            if score < self.best_score - self.min_delta:
                self.best_score = score
                self.best_epoch = epoch
                self.counter = 0
            else:
                self.counter += 1
        
        if self.counter >= self.patience:
            self.early_stop = True
        
        return self.early_stop


# === PREPROCESSING (applied once and cached) ===
# Only deterministic, spatial transforms - stores uint8 tensors (saves memory!)
preprocess_transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(config["upscale_size"]),  # Upscale from 32x32 to 128x128
])

# === RUNTIME AUGMENTATION (applied at each epoch) ===
# Random transforms for training (includes normalization at the end)
train_runtime_transforms = v2.Compose([
    v2.RandomCrop(config["upscale_size"], padding=4),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    v2.RandomRotation(15),
    v2.ToDtype(torch.float32, scale=True),  # Convert to float [0,1]
    v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # CIFAR-100 stats
    v2.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3))
])

# Test set: only normalization needed (spatial transforms already done)
test_runtime_transforms = v2.Compose([
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Load raw datasets
print("Loading datasets...")
train_set_raw = CIFAR100_noisy_fine('/kaggle/input/fii-atnn-2025-project-noisy-cifar-100/fii-atnn-2024-project-noisy-cifar-100', download=False, train=True, transform=None)
test_set_raw = CIFAR100_noisy_fine('/kaggle/input/fii-atnn-2025-project-noisy-cifar-100/fii-atnn-2024-project-noisy-cifar-100', download=False, train=False, transform=None)

# Cache raw PIL images (fast, lightweight)
train_set_cached = SimpleCachedDataset(train_set_raw)
test_set_cached = SimpleCachedDataset(test_set_raw)

# Preprocess and cache as tensors (done once!)
print("\n[TRAIN SET]")
train_set_preprocessed = PreprocessedDataset(train_set_cached, preprocess_transforms)
print("\n[TEST SET]")
test_set_preprocessed = PreprocessedDataset(test_set_cached, preprocess_transforms)

refinement_map = get_refinement_metadata(train_set_raw, device)

# Add runtime augmentations (applied each epoch for train, none for test)
train_set = RefinedAugmentationWrapper(train_set_preprocessed, train_runtime_transforms, refinement_map=refinement_map)
test_set = RefinedAugmentationWrapper(test_set_preprocessed, test_runtime_transforms)

print(f"\nTrain set ready: {len(train_set)} samples (with runtime augmentation)")
print(f"Test set ready: {len(test_set)} samples (fully cached)\n")

train_loader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True, pin_memory=pin_memory,num_workers=2,persistent_workers=True)
test_loader = DataLoader(test_set, batch_size=500, pin_memory=pin_memory,num_workers=2,persistent_workers=True)

# Load ResNet18 pretrained on ImageNet
print(f"Loading model: {config['model']} (pretrained on {config['pretrained']})")
model = timm.create_model(config["model"], pretrained=True, num_classes=100)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}\n")

# Label smoothing helps with noisy labels
criterion = nn.CrossEntropyLoss(label_smoothing=config["label_smoothing"])

# Create optimizer based on config
if config["optimizer"].lower() == "adamw":
    optimizer = optim.AdamW(
        model.parameters(),
        lr=config["lr"],
        weight_decay=config["weight_decay"],
        fused=True
    )
    print(f"Optimizer: AdamW (lr={config['lr']}, weight_decay={config['weight_decay']})")
elif config["optimizer"].lower() == "sgd":
    optimizer = optim.SGD(
        model.parameters(), 
        lr=config["lr"],
        momentum=config["momentum"],
        weight_decay=config["weight_decay"],
        nesterov=config["nesterov"],
        fused=True
    )
    print(f"Optimizer: SGD (lr={config['lr']}, momentum={config['momentum']}, weight_decay={config['weight_decay']}, nesterov={config['nesterov']})")
else:
    raise ValueError(f"Unknown optimizer: {config['optimizer']}. Supported: 'sgd', 'adamw'")

# Learning rate scheduler
if config["scheduler"] == "steplr":
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=config.get("step_size", 30),
        gamma=config.get("gamma", 0.1)
    )
    print(f"Scheduler: StepLR (step_size={config.get('step_size', 30)}, gamma={config.get('gamma', 0.1)})")

elif config["scheduler"] == "cosine":
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=config["epochs"],
        eta_min=config.get("cosine_eta_min", 1e-6)
    )
    print(f"Scheduler: CosineAnnealingLR (T_max={config['epochs']}, eta_min={config['cosine_eta_min']})")

elif config["scheduler"] == "warm_restarts":
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=config["warm_restarts_T0"],         
        T_mult=config.get("warm_restarts_mult", 1), 
        eta_min=config.get("cosine_eta_min", 1e-6)
    )
    # Highlight the per-batch setting so you know it's active
    step_mode = "Per-Batch" if config.get("scheduler_step_per_batch") else "Per-Epoch"
    print(f"Scheduler: WarmRestarts (T_0={config['warm_restarts_T0']}, T_mult={config['warm_restarts_mult']}, Step: {step_mode})")

else:
    scheduler = None
    print("Scheduler: None")

# === CUTMIX HELPER FUNCTION ===
def rand_bbox(size, lam):
    """Generates a random bounding box for CutMix."""
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

loss_threshold = config["loss_threshold"]

def train(epoch):
    print(f"\nEpoch {epoch+1}/{config['epochs']}")
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0
    global loss_threshold
    
    initial_batch_count = 0 
    use_cutmix = epoch >= config["switch_epoch"]
    aug_mode = "CutMix" if use_cutmix else "MixUp"
    
    # Note: Updated unpack to include target_a, target_b, and lam_target
    for batch_idx, (inputs, t_a, t_b, lam_t, weights) in enumerate(train_loader):
        inputs = inputs.to(device, non_blocking=True)
        t_a = t_a.to(device, non_blocking=True)
        t_b = t_b.to(device, non_blocking=True)
        lam_t = lam_t.unsqueeze(1).to(device, non_blocking=True) # [B, 1]
        weights = weights.to(device, non_blocking=True)
        
        initial_batch_count += inputs.size(0)

        # 1. Dynamic Filtering (Optional, keep for extreme outliers)
        if epoch >= config["warmup_epochs"]:
            with torch.no_grad():
                with torch.autocast(device.type, enabled=enable_half):
                    raw_outputs = model(inputs)
                    # Use t_a as primary for loss filtering
                    sample_losses = F.cross_entropy(raw_outputs, t_a, reduction='none')
                mask = sample_losses < loss_threshold
            if mask.sum() < 2: continue
            inputs, t_a, t_b, lam_t, weights = inputs[mask], t_a[mask], t_b[mask], lam_t[mask], weights[mask]

        # 2. Augmentation (MixUp/CutMix)
        rand_index = torch.randperm(inputs.size(0)).to(device)
        lam_aug = np.random.beta(config["aug_alpha"], config["aug_alpha"])

        if use_cutmix:
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam_aug)
            lam_aug = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
        else:
            inputs = lam_aug * inputs + (1 - lam_aug) * inputs[rand_index, :]

        # 3. Forward & Soft-Loss
        with torch.autocast(device.type, enabled=enable_half):
            outputs = model(inputs)
            
            # Combine the two internal soft targets (Original vs Refurbished)
            # Loss = lam_aug * (lam_t*loss(t_a) + (1-lam_t)*loss(t_b)) + (1-lam_aug)*MixedSamples...
            def get_soft_loss(out, target_a, target_b, l_t):
                return l_t.view(-1) * criterion(out, target_a) + (1 - l_t.view(-1)) * criterion(out, target_b)

            loss_current = get_soft_loss(outputs, t_a, t_b, lam_t)
            loss_shuffled = get_soft_loss(outputs, t_a[rand_index], t_b[rand_index], lam_t[rand_index])
            
            mixed_loss = lam_aug * loss_current + (1 - lam_aug) * loss_shuffled
            
            # Apply k-NN weights and mean
            batch_weights = lam_aug * weights + (1 - lam_aug) * weights[rand_index]
            loss = (mixed_loss * batch_weights).mean()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        if config.get("scheduler_step_per_batch") and scheduler is not None:
            scheduler.step(epoch + batch_idx / len(train_loader))

        running_loss += loss.item() * inputs.size(0)
        total += t_a.size(0)
        correct += outputs.argmax(1).eq(t_a).sum().item()

    if epoch >= config["warmup_epochs"]:
        loss_threshold *= config["dynamic_threshold_decay"]
    
    epoch_acc = 100.0 * correct / total
    keep_rate = (total / initial_batch_count) * 100
    print(f"Keep Rate: {keep_rate:.2f}% | Threshold: {loss_threshold:.4f} | Mode: {aug_mode}")
    return running_loss / total, epoch_acc, aug_mode

@torch.inference_mode()
def val():
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    for inputs, t_a, t_b, lam_t, _ in test_loader:
        inputs, targets = inputs.to(device), t_a.to(device)
        with torch.autocast(device.type, enabled=enable_half):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        running_loss += loss.item() * inputs.size(0)
        predicted = outputs.argmax(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

@torch.inference_mode()
def inference():
    model.eval()
    
    labels = []
    
    for inputs, _, _, _, _ in test_loader:
        inputs = inputs.to(device, non_blocking=True)
        with torch.autocast(device.type, enabled=enable_half):
            outputs = model(inputs)

        predicted = outputs.argmax(1).tolist()
        labels.extend(predicted)
    
    return labels

# Initialize WandB
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"cifar100noisy_{config['model']}_{config['optimizer']}_lr{config['lr']}_bs{config['batch_size']}_{timestamp}"

# wandb.init(
#     project=config["wandb_project"],
#     name=run_name,
#     config=config
# )

best = 0.0
best_epoch = 0

# Initialize early stopping
early_stopping = EarlyStopping(
    patience=config["early_stop_patience"],
    min_delta=config["early_stop_min_delta"],
    mode=config["early_stop_mode"]
)

print(f"\n{'='*70}")
print(f"Starting Training - {config['epochs']} epochs")
print(f"Model: {config['model']} (pretrained on {config['pretrained']})")
print(f"Optimizer: {config['optimizer'].upper()}, LR: {config['lr']}, Batch Size: {config['batch_size']}")
if config["optimizer"].lower() == "sgd":
    print(f"Momentum: {config['momentum']}, Nesterov: {config['nesterov']}")
print(f"Weight Decay: {config['weight_decay']}, Label Smoothing: {config['label_smoothing']}")
print(f"Scheduler: {config['scheduler']}")
print(f"Early Stopping: Enabled (patience={config['early_stop_patience']}, mode={config['early_stop_mode']})")
print(f"{'='*70}\n")

with tqdm(range(config["epochs"])) as tbar:
    for epoch in tbar:
        # 1. Run Train (includes internal per-batch scheduler steps)
        train_loss, train_acc, mode = train(epoch)
        
        # 2. Run Validation
        val_loss, val_acc = val()
        
        # 3. Handle LR logging (step only if NOT per-batch)
        if scheduler is not None:
            if not config.get("scheduler_step_per_batch"):
                scheduler.step()
            current_lr = scheduler.get_last_lr()[0]
        else:
            current_lr = config["lr"]
        
        # 4. Checkpointing
        if val_acc > best:
            best = val_acc
            best_epoch = epoch
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'val_loss': val_loss,
            }
            if scheduler is not None:
                checkpoint['scheduler_state_dict'] = scheduler.state_dict()
            torch.save(checkpoint, './best_model.pth')
        
        # 5. Update Progress Bar & Console
        status = f"Epoch {epoch+1}/{config['epochs']} | Train: {train_acc:.2f}% | Val: {val_acc:.2f}% | Best: {best:.2f}% | LR: {current_lr:.6f}"
        tbar.set_description(status)
        print(status)
        
        # 6. Early stopping check
        if early_stopping(val_acc, epoch):
            print(f"\n{'='*60}")
            print(f"Early stopping triggered at epoch {epoch+1}")
            print(f"Best Val Accuracy: {best:.2f}% at epoch {best_epoch+1}")
            print(f"{'='*60}\n")
            break
    

print(f"\n{'='*60}")
print(f"Training Complete!")
print(f"Best Val Accuracy: {best:.2f}% at epoch {best_epoch+1}")
print(f"Loading best model for inference...")
print(f"{'='*60}\n")

# Load best model for inference
checkpoint = torch.load('./best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Best model loaded (Epoch {checkpoint['epoch']+1}, Val Acc: {checkpoint['val_acc']:.2f}%)\n")

# Generate submission
data = {
    "ID": [],
    "target": []
}

print("Generating predictions...")
for i, label in enumerate(inference()):
    data["ID"].append(i)
    data["target"].append(label)

df = pd.DataFrame(data)
df.to_csv("/kaggle/working/submission.csv", index=False)

# Log final results to WandB
# wandb.summary["final_best_val_acc"] = best
# wandb.summary["best_epoch"] = best_epoch
# wandb.summary["total_epochs"] = config["epochs"]
# if scheduler is not None:
#     wandb.summary["final_lr"] = scheduler.get_last_lr()[0]
# else:
#     wandb.summary["final_lr"] = config["lr"]
# wandb.summary["early_stopped"] = early_stopping.early_stop
# wandb.summary["epochs_trained"] = best_epoch + 1 if early_stopping.early_stop else config["epochs"]

print(f"\n{'='*60}")
print(f"Submission saved to: ./submission.csv")
print(f"Best model saved to: ./best_model.pth")
print(f"Best Val Accuracy: {best:.2f}% (Epoch {best_epoch+1})")
if early_stopping.early_stop:
    print(f"Training stopped early (patience reached)")
print(f"{'='*60}\n")

# Finish WandB run
# 

Using device: cuda
Loading datasets...

[TRAIN SET]
Preprocessing 50000 images (this happens once)...


Cached 50000 preprocessed images

[TEST SET]
Preprocessing 10000 images (this happens once)...


Cached 10000 preprocessed images

Extracting Features for Analysis...


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Analyzing Data: 100%|██████████| 50000/50000 [03:45<00:00, 221.69it/s]


...

Epoch 82/100 | Train: 62.80% | Val: 70.52% | Best: 71.88% | LR: 0.000084

Epoch 83/100
Keep Rate: 73.58% | Threshold: 2.0807 | Mode: CutMix


Epoch 83/100 | Train: 63.66% | Val: 70.71% | Best: 71.88% | LR: 0.000065:  83%|████████▎ | 83/100 [5:11:34<1:04:55, 229.16s/it]

Epoch 83/100 | Train: 63.66% | Val: 70.71% | Best: 71.88% | LR: 0.000065

Epoch 84/100
Keep Rate: 73.66% | Threshold: 2.0703 | Mode: CutMix


Epoch 84/100 | Train: 64.89% | Val: 70.66% | Best: 71.88% | LR: 0.000049:  83%|████████▎ | 83/100 [5:15:28<1:04:36, 228.06s/it]
/tmp/ipykernel_24/4087130994.py:611: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues re

Epoch 84/100 | Train: 64.89% | Val: 70.66% | Best: 71.88% | LR: 0.000049

Early stopping triggered at epoch 84
Best Val Accuracy: 71.88% at epoch 49


Training Complete!
Best Val Accuracy: 71.88% at epoch 49
Loading best model for inference...

Best model loaded (Epoch 49, Val Acc: 71.88%)

Generating predictions...

Submission saved to: ./submission.csv
Best model saved to: ./best_model.pth
Best Val Accuracy: 71.88% (Epoch 49)
Training stopped early (patience reached)

